In [1]:
import sqlite3

con = sqlite3.connect("./DB/stockPrice.db")
cur  = con.cursor()

create_table_sql ="""
CREATE TABLE stockPrice (code TEXT,date TEXT, open REAL, high REAL, low REAL, close REAL,adjclose REAL,volume REAL,PRIMARY KEY(code, date))
"""

cur.execute(create_table_sql)

In [6]:
import pandas as pd
import yfinance


def getStockPriceInfo(code:str,period:str,interval:str) -> pd.core.frame.DataFrame:

    # DataFrameを作成します
    table_name = "stockPrice"
    df = yfinance.download(str(code) + ".T", period=period,interval=interval)

    #indexを最初の列に挿入
    df.reset_index(inplace= True)
    df = df.rename(columns={'index': 'date'})
    #codeを最初の列に挿入
    df.insert(0, 'code', str(code))

    #列名の変更
    df.columns = ['code', 'date', 'open', 'high', 'low', 'close', 'adjclose', 'Volume']

    #'date'列をdatetime->strへ型変換
    df['date'] = df['date'].astype(str)

    return df

df = getStockPriceInfo(code='130A',period='1mo',interval='1d')

[*********************100%%**********************]  1 of 1 completed


In [7]:
df

,code,date,open,high,low,close,adjclose,Volume
0,130A,2024-03-04,2093.0,2155.0,2020.0,2072.0,2072.0,95300
1,130A,2024-03-05,2132.0,2192.0,2001.0,2060.0,2060.0,192500
2,130A,2024-03-06,2060.0,2326.0,2019.0,2272.0,2272.0,219800
3,130A,2024-03-07,2422.0,2448.0,2136.0,2190.0,2190.0,212200
4,130A,2024-03-08,2240.0,2280.0,2082.0,2105.0,2105.0,89900
5,130A,2024-03-11,2000.0,2005.0,1651.0,1708.0,1708.0,311800
6,130A,2024-03-12,1701.0,1980.0,1686.0,1925.0,1925.0,207300
7,130A,2024-03-13,1901.0,1977.0,1809.0,1858.0,1858.0,71600
8,130A,2024-03-14,1818.0,1858.0,1756.0,1790.0,1790.0,68600
9,130A,2024-03-15,1764.0,1770.0,1567.0,1672.0,1672.0,188700


In [9]:
#connection確立
con = sqlite3.connect("./DB/stockPrice.db")

#一括書込み
df.to_sql('stockPrice',con,if_exists='append',index=None)

#commit
con.commit()

In [11]:
con = sqlite3.connect("./DB/stockPrice.db")

sql = '''
 SELECT date FROM stockprice 
'''
res = con.execute(sql)
res.fetchall()

[('2024-03-04',),
 ('2024-03-05',),
 ('2024-03-06',),
 ('2024-03-07',),
 ('2024-03-08',),
 ('2024-03-11',),
 ('2024-03-12',),
 ('2024-03-13',),
 ('2024-03-14',),
 ('2024-03-15',),
 ('2024-03-18',),
 ('2024-03-19',),
 ('2024-03-21',),
 ('2024-03-22',),
 ('2024-03-25',),
 ('2024-03-26',),
 ('2024-03-27',),
 ('2024-03-28',),
 ('2024-03-29',),
 ('2024-04-01',),
 ('2024-04-02',)]

In [24]:
import pandas as pd
import re
import pandas as pd
import urllib.request

#セキュリティ的にだめ！！（一時凌ぎ）
#https://qiita.com/shutokawabata0723/items/9733a7e640a175c23f39
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

def getStockCodeDataFrame() -> pd.core.frame.DataFrame:

    URL = 'https://www.jpx.co.jp/markets/statistics-equities/misc/01.html'


    response = urllib.request.urlopen(URL).read().decode("utf-8")
    string_html = re.findall('<a href=\".+?\.xls\"',response)
    url_list = []
    for i in string_html:
        j = i.lstrip('<a href=\"')
        k = j.rstrip('\"')
        url_list.append('https://www.jpx.co.jp'+k)
    url = url_list[0]

    #国内株式のみ抽出

    df = pd.read_excel(url)
    df = df[(df.iloc[:,3] == 'プライム（内国株式）' ) | (df.iloc[:,3] == 'スタンダード（内国株式）' ) | (df.iloc[:,3] == 'グロース（内国株式）' ) ]

    #列名の変更
    df.columns = ['date', 'code', 'office_name', 'market_class', 'industry_detail_code', 'industry_detail', 'industry_code', 'industry','scale_code','scale_class']

    #'date'列をdatetime->strへ型変換
    df['date'] = df['date'].astype(str)
    df['code'] = df['code'].astype(str)
    df['industry_detail_code'] = df['industry_detail_code'].astype(str)
    df['industry_code'] = df['industry_code'].astype(str)
    df['scale_code'] = df['scale_code'].astype(str)

    return df

In [25]:
codes = getStockCodeDataFrame()
codes

,date,code,office_name,market_class,industry_detail_code,industry_detail,industry_code,industry,scale_code,scale_class
0,20240229,1301,極洋,プライム（内国株式）,50,水産・農林業,1,食品,7,TOPIX Small 2
5,20240229,130A,Ｖｅｒｉｔａｓ Ｉｎ Ｓｉｌｉｃｏ,グロース（内国株式）,3250,医薬品,5,医薬品,-,-
19,20240229,1332,ニッスイ,プライム（内国株式）,50,水産・農林業,1,食品,4,TOPIX Mid400
20,20240229,1333,マルハニチロ,プライム（内国株式）,50,水産・農林業,1,食品,4,TOPIX Mid400
31,20240229,135A,ＶＲＡＩＮ Ｓｏｌｕｔｉｏｎ,グロース（内国株式）,5250,情報・通信業,10,情報通信・サービスその他,-,-
...,...,...,...,...,...,...,...,...,...,...
4338,20240229,9993,ヤマザワ,スタンダード（内国株式）,6100,小売業,14,小売,7,TOPIX Small 2
4339,20240229,9994,やまや,スタンダード（内国株式）,6100,小売業,14,小売,7,TOPIX Small 2
4340,20240229,9995,グローセル,プライム（内国株式）,6050,卸売業,13,商社・卸売,7,TOPIX Small 2
4341,20240229,9996,サトー商会,スタンダード（内国株式）,6050,卸売業,13,商社・卸売,-,-


In [31]:
import sqlite3

con = sqlite3.connect("./DB/stockCode.db")
cur  = con.cursor()

create_table_sql ="""
CREATE TABLE stockCode 
(date TEXT,code TEXT, brand TEXT, market TEXT, 
 industry_detail TEXT,industry_detail_code Text,
 industry TEXT,industry_code Text,scale_code Text,scale_class Text
 ,PRIMARY KEY(code))
"""

cur.execute(create_table_sql)
con.close()

In [34]:
#connection確立
con = sqlite3.connect("./DB/stockCode.db")

#一括書込み
df = getStockCodeDataFrame()
df.to_sql('stockCode',con,if_exists='replace',index=None)

#commit
con.commit()

DatabaseError: Execution failed on sql 'DROP TABLE "stockCode"': database is locked

In [33]:
con = sqlite3.connect("./DB/stockCode.db")

sql = '''
 SELECT * FROM stockCode 
'''
res = con.execute(sql)
res.fetchone()

('20240229',
 '1301',
 '極洋',
 'プライム（内国株式）',
 '50',
 '水産・農林業',
 '1',
 '食品',
 '7',
 'TOPIX Small 2')